# Ising in 2D

In [ ]:
using Printf #string formatting
using Random # random number utilities
using Plots # plotting
using Statistics # statistical tools
using Distributions # useful distributions
using LaTeXStrings
using LinearAlgebra

In [ ]:
# for readability, these are good settings to use
default(xtickfontsize=14,  ytickfontsize=14, ztickfontsize=14,
    guidefontsize=14, legendfontsize=12, lw=2,ms=8)

## Setup Energy and Proposal
The energy, with $J=1$, is
$$
E(x) = - \sum_{i} \sum_{j\in N_i} \sigma_i \sigma_{j}
$$
where $N_i$ is the neighbor set of lattice site $i$.  

In [ ]:
# energy

# construct set of neighbors to (i,j) on an LxL lattice
function N(i,j,L)
    neighbors = [];
    
    if(i<L)
        push!(neighbors,[i+1,j])
    else
        push!(neighbors,[1,j])
    end
    if(i>1)
        push!(neighbors,[i-1,j])
    else
        push!(neighbors,[L,j])
    end
    
    if(j<L)
        push!(neighbors,[i,j+1])
    else
        push!(neighbors,[i,1])
    end
    if(j>1)
        push!(neighbors,[i,j-1])
    else
        push!(neighbors,[i,L])
    end
    
    return neighbors
    
end

function E(x)
    
    L, = size(x);
    
    eng = 0.0;
    
    for i in 1:L, j in 1:L
        neighbors = N(i,j,L);
        for neighbor in neighbors
            eng += -0.5 * x[i,j] * x[neighbor[1], neighbor[2]];
        end
    end
    return eng;
end

# proposal
function g(x)
    L, = size(x);
    y = copy(x);
    ξ = rand(DiscreteUniform(1,L),2);
    y[ξ[1], ξ[2]] = -y[ξ[1], ξ[2]];
    return y;
end

#acceptance probability
function h(x, y, β)
    return min(1, exp(β * E(x)-β* E(y))); 
end


In [ ]:
L = 16; # number of lattice sites
# β = 1/100; # inverse temperature
β = 2; # inverse temperature


Random.seed!(100)
# random initial condition
X₀ = 2*rand(DiscreteUniform(0,1),L,L) .- 1;
# deterministic initial condition
#X₀ = ones(L,L);

X = copy(X₀);
X_vals = [copy(X₀)];
Y = similar(X);

n = 10^4;

for j in 1:n-1
    Y .= g(X); 
    U = rand();
    if (U< h(X, Y, β))
        X .= Y;
#         println("exchange");
#         display(Y);
    end
    push!(X_vals, copy(X))
end


In [ ]:
X_vals[4]

In [ ]:
X = X_vals[end];

up = findall(X.>0);
down = findall(X.<0);
scatter(getindex.(up,1),getindex.(up,2),label="",marker=:+)
scatter!(getindex.(down,1),getindex.(down,2),label="",marker=:hline)

In [ ]:
# create an animation of the spin evolution

anim = @animate for i=1:10:10000
    X = X_vals[i]
    
    up = findall(X.>0);
    down = findall(X.<0);
    scatter(getindex.(up,1),getindex.(up,2),label="",marker=:+)
    scatter!(getindex.(down,1),getindex.(down,2),label="",marker=:hline)    
    title!(@sprintf("n = %d",i-1))
end



In [ ]:
gif(anim, fps=10)

In [ ]:
# compute energy
E_vals = [E(x) for x in X_vals];
plot(0:n-1, E_vals, label="Energy",lw=2)
plot!(0:n-1, cumsum(E_vals) ./ (1:n), label="Running Avg.",lw=2)
xlabel!(L"$t$")
title!(LaTeXString(@sprintf("\$L = %d,\\; \\beta = %g\$", L, β)))

In [ ]:
# compute mean magnetization
M_vals = [mean(x) for x in X_vals];
plot(0:n-1, M_vals, label="Magnetization",lw=2)
plot!(0:n-1, cumsum(M_vals) ./ (1:n), label="Running Avg.",lw=2)
xlabel!(L"$t$")
title!(LaTeXString(@sprintf("\$L = %d,\\; \\beta = %g\$", L, β)))

In [ ]:
# energy and spin vs. temperature


L = 4; # number of lattice sites
β_vals = 10 .^(-1:0.2:1);

M_vals = Float64[];
E_vals =  Float64[];

Random.seed!(100)
# random initial condition
X₀ = 2*rand(DiscreteUniform(0,1),L,L) .- 1;
# deterministic initial condition
#X₀ = ones(L,L);
Y = similar(X₀);

n = 10^4;

Random.seed!(100)
for β in β_vals
    X = copy(X₀);
    E_avg = E(X)/n;
    M_avg = mean(X)/n;
    for j in 1:n-1
        Y .= g(X); 
        U = rand();
        if (U< h(X, Y, β))
            X .= Y;
        end
        E_avg += E(X)/n;
        M_avg += mean(X)/n;
    end
    
    push!(E_vals, E_avg);
    push!(M_vals, M_avg);    
    
end



In [ ]:
scatter(1 ./β_vals, E_vals, label="Numerical Estimates", xscale=:log10)
#xlabel!(L"$\beta$")
xlabel!("Temperature")
ylabel!(L"$\langle E\rangle$")

In [ ]:
scatter(1 ./β_vals, abs.(M_vals),label="Numerical Estimates", xscale=:log10)
#xlabel!(L"$\beta$")
xlabel!("Temperature")
ylabel!(L"$|\langle M\rangle|$")